### Tutorial - 9 : Power Assignment in a wireless communication system



### Variables in our Linear Fractional Problem : -

$p_i$ $\rightarrow$ powers transmitting to $n$ receivers

$G$ $→$ Path Gains matrix (matrix of dimension $n \times n$)

$G_{ij}$ $\rightarrow$ Path gain from transmitter $j$ to receiver $i$

$S_i$ $\rightarrow$ Signal power at receiver $i$

$I_i$ $\rightarrow$ Interference power at receiver $i$, $I_i = ΣG_{ik}P_k$ (here, $i \neq k$)

SINR $=$ $\cfrac{S_i}{I_i + σ_i}$ 

### Constraints in our Problem 

$p_i \geq 0$

$p_i \leq P_{max}$ 

$\Sigma_{k \in K_l}$ ${p_k} \leq P^{gp}_l$

$\Sigma$ $G_{ik} p_{k} \leq P^{rc}_{i}, i = 1, 2,...., n$

## Imports 

In [ ]:
import numpy as npy 
import cvxpy as cvpy

## Given Data 

In [ ]:
# Number of transmitters 
n = 5

# Path gains matrix 
G = npy.array([[1.0, 0.1, 0.2, 0.1, 0.0],
     [0.1, 1.0, 0.1, 0.1, 0.0],
     [0.2, 0.1, 2.0, 0.2, 0.2],
     [0.1, 0.1, 0.2, 1.0, 0.1],
     [0.0, 0.0, 0.2, 0.1, 1.0]])

# maximum individual transmitter power 
P_max = 3

# maximum individual receiving power 
P_received = 5

# Maximum noise levels
noise = 0.5

# We take groups as boolean values (0 or 1), because groups are disjoint
# If a transmitter is present in a particular group, it has a value of 1 
groups = npy.array([[1, 1, 0, 0, 0], [0, 0, 1, 1, 1]])

g_size = 2

# maximum group power for each group
max_grp_power = npy.array([4,6])

### Solving the optimization problem 

In [ ]:
p = cvpy.Variable(n)

# leaving out the multiplication with p, added in constraints 

G_ii = npy.diag(G)

S = cvpy.multiply((G_ii.T), p)
I = G@p - S

In [ ]:
# Bisection algorithm parameters 

alpha = cvpy.Parameter(1)

L_limit = 0
U_limit = int(1e4)

In [ ]:
# precision of alpha
precision = 0.05

# objective and constraints 
objective = cvpy.Minimize(alpha)

constraints = [p <= P_max, p >= 0, G @ p <= P_received]

p_subset_sum = [0, 0]

for i in range(g_size): 
  for j in range(n):
    if(groups[i][j] != 0):
      p_subset_sum[i] += p[j]

  constraints += [p_subset_sum[i] <= max_grp_power[i]]

constraints += [I + noise <= alpha*(S)]

# Implementing the bisection algorithm

while(U_limit - L_limit > precision):
  alpha.value = npy.atleast_1d((L_limit + U_limit) / 2)

  problem = cvpy.Problem(objective, constraints)
  problem.solve()

  # checking feasibility 
  if problem.status == 'optimal':
    U_limit = alpha.value
    temp_p = p.value
  else:
    L_limit = alpha.value

print("Maximum SINR value is : ", 1/alpha.value)
print("The values of powers are : ", temp_p)

Maximum SINR value is :  [1.74762667]
The values of powers are :  [2.05325577 1.85690341 1.5972308  2.3231306  1.85569467]
